In [ ]:
# decision_tree_assignment.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

In [ ]:
# === PATHS ===
base_path = r"D:\DATA SCIENCE\ASSIGNMENTS\13 decision tree\Decision Tree"
file_path = os.path.join(base_path, "heart_disease.xlsx")

In [ ]:
# === LOAD DATA ===
df = pd.read_excel(file_path, sheet_name="Heart_disease")

In [ ]:
# === CLEANING ===
# Convert boolean to int
for col in ['fbs', 'exang']:
    if df[col].dtype == bool:
        df[col] = df[col].astype(int)
    elif df[col].dtype == object:
        df[col] = df[col].map({'True':1, 'TURE':1, 'FALSE':0, 'False':0})  # fix odd strings

In [ ]:
# Sex mapping
df['sex'] = df['sex'].map({'Male':1, 'Female':0})

In [ ]:
# Target binary (num>0 → 1)
df['target'] = df['num'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
# One-hot encode categorical vars
ohe_cols = ['cp','restecg','slope','thal']
df_processed = pd.get_dummies(df, columns=ohe_cols, drop_first=True)

In [ ]:
# Drop original num column
df_processed = df_processed.drop(columns=['num'])

In [ ]:
# === TRAIN/TEST SPLIT ===
X = df_processed.drop(columns=['target'])
y = df_processed['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
# === BASELINE MODEL ===
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:,1]

In [ ]:
# === EVALUATION ===
metrics = {
    "Accuracy": accuracy_score(y_test, y_pred),
    "Precision": precision_score(y_test, y_pred),
    "Recall": recall_score(y_test, y_pred),
    "F1-score": f1_score(y_test, y_pred),
    "ROC-AUC": roc_auc_score(y_test, y_proba)
}

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
# === SAVE RESULTS ===
# Processed dataset
processed_path = os.path.join(base_path, "heart_processed.csv")
df_processed.to_csv(processed_path, index=False)

In [ ]:
# Model
model_path = os.path.join(base_path, "decision_tree_model.pkl")
joblib.dump(clf, model_path)

In [ ]:
# Metrics report
report_path = os.path.join(base_path, "decision_tree_report.txt")
with open(report_path, "w") as f:
    f.write("=== Decision Tree Evaluation ===\n")
    for k,v in metrics.items():
        f.write(f"{k}: {v:.4f}\n")
    f.write("\nClassification Report:\n")
    f.write(classification_report(y_test, y_pred))

In [ ]:
# Confusion matrix plot
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
cm_path = os.path.join(base_path, "confusion_matrix.png")
plt.savefig(cm_path)
plt.close()

In [ ]:
# Decision Tree visualization
plt.figure(figsize=(20,10))
plot_tree(clf, feature_names=X.columns, class_names=["No Disease","Disease"],
          filled=True, rounded=True, fontsize=8)
tree_path = os.path.join(base_path, "decision_tree.png")
plt.savefig(tree_path)
plt.close()

In [ ]:
print("All done! Files saved in:", base_path)
print("Processed CSV:", processed_path)
print("Model:", model_path)
print("Report:", report_path)
print("Plots:", cm_path, "and", tree_path)

In [ ]:
# eda_heart_disease.py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# === PATHS ===
base_path = r"D:\DATA SCIENCE\ASSIGNMENTS\13 decision tree\Decision Tree"
file_path = os.path.join(base_path, "heart_disease.xlsx")

In [ ]:
# === LOAD DATA ===
df = pd.read_excel(file_path, sheet_name="Heart_disease")

In [ ]:
# === BASIC INFO ===
print("Shape:", df.shape)
print("\n--- Info ---")
print(df.info())
print("\nMissing values per column:\n", df.isnull().sum())
print("\nDuplicate rows:", df.duplicated().sum())
print("\nDescriptive statistics:\n", df.describe().T)

In [ ]:
# === TARGET DISTRIBUTION ===
print("\nTarget distribution:\n", df['num'].value_counts())

In [ ]:
# === HISTOGRAMS ===
numeric_cols = ['age','trestbps','chol','thalch','oldpeak']
df[numeric_cols].hist(bins=20, figsize=(12,8))
plt.suptitle("Histograms of Numeric Features")
plt.savefig(os.path.join(base_path, "histograms.png"))
plt.close()

In [ ]:
# === BOXPLOTS ===
plt.figure(figsize=(12,8))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(2,3,i)
    sns.boxplot(y=df[col])
    plt.title(f"Boxplot of {col}")
plt.tight_layout()
plt.savefig(os.path.join(base_path, "boxplots.png"))
plt.close()

In [ ]:
# === CORRELATION MATRIX ===
corr = df[numeric_cols + ['num']].corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0)
plt.title("Correlation Matrix")
plt.savefig(os.path.join(base_path, "correlation_matrix.png"))
plt.close()

In [ ]:
print("\nEDA completed. Plots saved in:", base_path)
print("- histograms.png")
print("- boxplots.png")
print("- correlation_matrix.png")